In [1]:
import yfinance as yf
import matplotlib.pylab as plt
from PyEMD import EMD
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import keras


2024-05-17 05:39:12.573076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load data

In [ ]:
V = Load_data # data should be (N,) dims numpy array

In [4]:
emd = EMD()
imf = emd(V).T

In [ ]:
plt.figure(figsize=(5,5),dpi=400)
plt.style.use("tableau-colorblind10")
plt.plot(imf);
plt.legend()
plt.xlabel('Time')
plt.ylabel('Decomposed value')


# Training 

In [ ]:

time = np.linspace(0,1,V.shape[0]).reshape(V.shape[0],1)
dt = time[1,0]-time[0,0]

data = np.hstack((time,imf))


fi_2 = data[0:-2,1:]
fi_1 = data[1:-1,1:]
fi = data[2:,1:]
v = V[2:]
data = data[2:,:]
Imf = imf[2:,:]


y = (3*fi-4*fi_1+fi_2)/(2*dt)


## split data for training and testing ##
rat_train_data = .85

train_data_size = int(data.shape[0]*rat_train_data)
test_data_size = data.shape[0]-train_data_size

Model = []

for i in range(y.shape[1]):
    print(i)
    y[:,1].reshape(y.shape[0],1)
    X_train = data[0:train_data_size,:]

    Y_train = y[1:train_data_size+1,i].reshape(train_data_size,1)
    
    dim = X_train.shape[1]
    
    model = Sequential()
    model.add(Dense(20, input_dim=dim, activation= "relu"))
    model.add(Dense(20, activation= "relu"))
    model.add(Dense(1))
    optimizer = keras.optimizers.Adam(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, Y_train ,epochs = 1000, verbose=1)
    
    Model.append(model)

# Testing

In [ ]:
rec1 = 0
for i in range(y.shape[1]):
    
    X_train = data[train_data_size:-1,:]
    
    y_pre = Model[i].predict(X_train)
    
    y_next = y_pre*dt+Imf[train_data_size:-1,i].reshape(test_data_size-1,1)
    rec1+= y_next
    print(np.mean(np.abs(y_next - Imf[train_data_size+1:,i].reshape(test_data_size-1,1))))

In [ ]:
plt.plot(rec1)
plt.plot(v[train_data_size+1:])
plt.show()
error_d = rec1.T-v[train_data_size+1:]
plt.hist(error_d.T)
print(np.mean(np.abs(error_d.T)))
print(np.var(np.abs(error_d.T)))

In [ ]:

X_train = np.array([data[train_data_size]])
pre_li=[]
for j in range(test_data_size):
    print('j: ',j)
    rec12 = 0
    X_new = X_train*0
    X_new[0,0]=X_train[0,0]+dt
    for i in range(y.shape[1]):

    #     X_train = data[train_data_size:-1,:]
        
        y_pre = Model[i].predict(X_train)
        
        
        
        y_next = y_pre*dt+X_train[0,i+1]
        X_new[0,i+1] = y_next
        rec12+= y_next
        print(np.mean(np.abs(y_next - Imf[train_data_size+1:,i].reshape(test_data_size-1,1))))
    X_train = X_new
    pre_li.append(rec12)
    
    
    

In [ ]:
p_l = []
for i in pre_li:
    p_l.append(i[0,0])


plt.plot(p_l)

plt.plot(v[train_data_size+1:])
plt.show()
error_d = np.array(p_l)[-test_data_size:]-v[train_data_size+1:][-test_data_size:]
plt.plot(error_d.T)